In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
from pathlib import Path
%load_ext autoreload
%autoreload 2


In [ ]:
# mouse = "mouse2"
# df = utils.load_example_data(mouse)

In [ ]:
import socket
socket.gethostname()

In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)


In [ ]:

# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)

In [ ]:
# download the data for a specific animal
mouse = "ACV007"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_session_data(
    project_name=tv_projects[1],
    animal=mouse,
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)

In [ ]:
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
import lecilab_behavior_analysis.plots as plots
import lecilab_behavior_analysis.df_transforms as dft
df = dft.fill_missing_data(df)

# add a column with the date for the day
df = dft.add_day_column_to_df(df)

# create a figure with 1 axis for the calendar plot
import matplotlib.pyplot as plt
fig, ax_cal = plt.subplots(figsize=(10, 5), dpi=300)
# generate the calendar plot
dates_df = dft.get_dates_df(df)
cal_image = plots.rasterize_plot(plots.training_calendar_plot(dates_df), dpi=300)
# paste the calendar plot filling the entire axis
ax_cal.imshow(cal_image)
ax_cal.axis("off")

plt.show()

In [ ]:
from lecilab_behavior_analysis.figure_maker import subject_progress_figure
fig = subject_progress_figure(df, perf_window=100, summary_matrix_plot=False)

In [ ]:
from lecilab_behavior_analysis.figure_maker import session_summary_figure
from lecilab_behavior_analysis.df_transforms import add_trial_of_day_column_to_df, add_day_column_to_df
# select the session you want to plot
date = "2025-05-06"
df = add_day_column_to_df(df)
df = add_trial_of_day_column_to_df(df)
sdf = df[df["year_month_day"] == date]
fig = session_summary_figure(sdf, mouse, perf_window=15, width=10, height=5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit
import df_transforms as dft
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
df_test = df.dropna(subset = ['visual_stimulus'])
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for i, ax in zip(df_test.groupby('current_training_stage').groups.keys(), [ax1, ax2]):
    df_test_sub = df_test[df_test["current_training_stage"] == i]
    df_test_sub['visual_stimulus_devi'] = df_test_sub['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
    # This was good in order to make the fit work for both left and right choices!
    df_test_sub['visual_stimulus_devi'] = df_test_sub.apply(
        lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
        axis=1
    )
    df_test_sub = dft.add_mouse_first_choice(df_test_sub)
    df_test_sub['left_choice_new'] = df_test_sub['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
    
    # Now we can fit the data and visualize the results
    X = df_test_sub['visual_stimulus_devi'].values.reshape(-1, 1)
    y = df_test_sub['left_choice_new'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)

    # Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
    # For plotting, we can generate a range of values for the visual stimulus deviation
    import numpy as np
    xs = np.linspace(df_test_sub['visual_stimulus_devi'].min(), df_test_sub['visual_stimulus_devi'].max(), 100).reshape(-1, 1)
    y_prob = model.predict_proba(xs)[:, 1]

    # Plot the actual choices of the mouse

    sns.pointplot(
        x='visual_stimulus_devi',
        y='left_choice_new',
        data=df_test_sub,
        estimator=lambda x: np.mean(x),
        color='blue',
        markers='o',
        errorbar=("ci", 95),
        ax=ax,
        label='Observed Choices',
        native_scale= True,
        linestyles='',
    )

    # overlay the fitted logistic regression curve
    ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
    ax.set_xlabel("Visual Stimulus Deviation")
    ax.set_ylabel("Probability of Left Choice")
    plt.title("Psychometric Curve")
    plt.legend()
plt.show()

In [ ]:
df_test = df.dropna(subset = ['visual_stimulus'])
df_test['visual_stimulus_devi'] = df_test['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
# This was good in order to make the fit work for both left and right choices!
df_test['visual_stimulus_devi'] = df_test.apply(
    lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
    axis=1
)
df_test = dft.add_mouse_first_choice(df_test)
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
# Now we can fit the data and visualize the results
X = df_test['visual_stimulus_devi'].values.reshape(-1, 1)
y = df_test['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)

# Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
# For plotting, we can generate a range of values for the visual stimulus deviation
import numpy as np
xs = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 100).reshape(-1, 1)
y_prob = model.predict_proba(xs)[:, 1]

# Plot the actual choices of the mouse
fig, ax = plt.subplots(figsize=(5, 5))
sns.pointplot(
    x='visual_stimulus_devi',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    ax=ax,
    label='Observed Choices',
    native_scale= True,
    linestyles='',
)

# overlay the fitted logistic regression curve
ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
ax.set_xlabel("Visual Stimulus Deviation")
ax.set_ylabel("Probability of Left Choice")
plt.title("Psychometric Curve")
plt.legend()
plt.show()

In [ ]:
df_test['visual_stimulus_diff'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
df_test['visual_stimulus_diff'] = df_test.apply(
    lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
    axis=1
)

In [ ]:
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)

# Now we can fit the data and visualize the results
X = df_test.sort_values(by='visual_stimulus_diff')['visual_stimulus_diff'].values.reshape(-1, 1)
y = df_test.sort_values(by='visual_stimulus_diff')['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)
y_prob = model.predict_proba(X)[:, 1]

In [ ]:
df_test = df.dropna(subset = ['visual_stimulus'])
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for i, ax in zip(df_test.groupby('current_training_stage').groups.keys(), [ax1, ax2]):
    df_test_sub = df_test[df_test["current_training_stage"] == i]
    df_test_sub['visual_stimulus_diff'] = df_test_sub['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
    # This was good in order to make the fit work for both left and right choices!
    df_test_sub['visual_stimulus_diff'] = df_test_sub.apply(
        lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
        axis=1
    )
    df_test_sub = dft.add_mouse_first_choice(df_test_sub)
    df_test_sub['left_choice_new'] = df_test_sub['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
    
    # Now we can fit the data and visualize the results
    X = df_test_sub.sort_values(by='visual_stimulus_diff')['visual_stimulus_diff'].values.reshape(-1, 1)
    y = df_test_sub.sort_values(by='visual_stimulus_diff')['left_choice_new'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)
    y_prob = model.predict_proba(X)[:, 1]

    # Plot the actual choices of the mouse

    sns.pointplot(
        x='visual_stimulus_diff',
        y='left_choice_new',
        data=df_test_sub,
        estimator=lambda x: np.mean(x),
        color='blue',
        markers='o',
        errorbar=("ci", 95),
        ax=ax,
        label='Observed Choices',
        native_scale= True,
        linestyles='',
    )

    # overlay the fitted logistic regression curve
    ax.plot(X, y_prob, color='red', label='Logistic Regression Fit')
    ax.set_xlabel("Visual Stimulus difference")
    ax.set_ylabel("Probability of Left Choice")
    plt.title("Psychometric Curve")
    plt.legend()
plt.show()

In [ ]:

df_test = dft.add_mouse_first_choice(df_test)
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)

# Now we can fit the data and visualize the results
X = df_test.sort_values(by='visual_stimulus_diff')['visual_stimulus_diff'].values.reshape(-1, 1)
y = df_test.sort_values(by='visual_stimulus_diff')['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)
y_prob = model.predict_proba(X)[:, 1]

# Plot the actual choices of the mouse
fig, ax = plt.subplots(figsize=(5, 5))
sns.pointplot(
    x='visual_stimulus_diff',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    ax=ax,
    label='Observed Choices',
    native_scale= True,
    linestyles='',
    scale = 0.2
)

# overlay the fitted logistic regression curve
ax.plot(X, y_prob, color='red', label='Logistic Regression Fit')
ax.set_xlabel("Visual Stimulus Difference")
ax.set_ylabel("Probability of Left Choice")
plt.title("Psychometric Curve")
plt.legend()
plt.show()
# It is interesting to compare the effects of the relative difference between the two visual stimuli,
# and the absolute difference between them.

# Maybe what we can do is to train another logistic regression model, adding as well the absolute difference
# between the two visual stimuli, and see how it affects the probability of a left choice.
# Do you know what I mean?
